In [1]:
import pandas as pd
import json
import requests
import random
from lxml import html
import re

from  util.spider import Spider
from parsel import Selector #unused
from util.http_utility import get_http_headers
from util.user_agent import get_random_ua

# Country : Romania

Priority for scraping:
- total time
- list of ingredients
- list of instructions


Extra:
- prep time
- cook time
- number of servings
- category

In [ ]:
attrs_template = {
    #fill in the xpaths here
        'name': '',
        'total_time' : '',
        'ingredients': '',
        'instructions': '',
        'prep_time': '',
        'cook_time': '',
        'servings': '',
        'category': '',
}

In [2]:
attrs = {
        'name':         '//meta[@property="og:title"]/@content',
        'ingredients':  '//meta[@itemprop="recipeIngredient"]/@content',
        'total_time':   '//meta[@itemprop="totalTime"]/@content',
        'instructions': '//meta[@itemprop="recipeInstructions"]/@content',
        'servings':     '//meta[@itemprop="recipeYield"]/@content',
        'category':     '//meta[@itemprop="recipeCategory"]/@content',
        'prep_time':    '',
        'cook_time':    '',
}

listing={'items': '//div[@class="contentS"]/div[@class="contentTxt"]//a/@href', 'next': { 'next_page_str': '?p={}', 'type': 'url'}}
seeds = {'https://www.bucataras.ro/retete-traditionale/romania/'}

In [3]:
#setup variables
user_agent = get_random_ua()
custom_header = { #setup custom header because romania requires certain headers
        'referer': 'https://www.google.com/',
        'Accept-Language': '*',
        'Accept-Encoding': '*',
        'Accept': '*',
        'user-agent': user_agent}
romania_spider = Spider('https://www.bucataras.ro', seeds= seeds, listing =listing,attrs= attrs, header=custom_header)
#print(romania_spider.attrs)
romania_spider.scrape_one_item('https://www.bucataras.ro/retete/tort-pajistea-cu-flori-5-ani-de-bucataras-96417.html') #try out scraping one item

{'name': 'Reteta Tort Pajistea cu flori / 5 ani de bucătăras',
 'total_time': 'PT120M',
 'ingredients': '\r\n Pentru blat: ',
 'instructions': ' Mai întâi se face blatul, cu o zi înainte. Se separă albușurile de gălbenușuri, se mixează cu un praf de sare extrafină și când încep să ia volum, se adaugă zahărul vanilinat și cel tos, puțin câte puțin și se continuă mixarea. Se adaugă apoi gălbenușurile, unul câte unul și la urmă, după omogenizare, se încorporează făina, având grijă să nu se lase compoziția. Se unge o formă de copt cu unt, se toarnă compoziția și se dă la cuptor, la foc mediu, până ce blatul crește și se rumenește frumos. Atunci se stinge focul și se întredeschide ușa de la cuptor. Se mai lasă acolo blatul câteva minute, după care se scoate și se lasă până a doua zi, când se feliază în trei straturi. \r\n Crema mousseline, care este delicioasă și fină, se face foarte ușor dintr-o cremă patiseră (cea pentru ecleruri) în care se adaugă unt. (Am folosit 150 grame de unt, dar d

Here below, we try to scrape only the first 5 pages:

In [4]:
result_list = romania_spider.start_scrape(max_pages=5) #try scraping 5 pages

spider is scraping page: 1
spider is scraping page: 2
spider is scraping page: 3
spider is scraping page: 4
spider is scraping page: 5


In [6]:
result_df = pd.DataFrame(result_list)

In [7]:
result_df

,name,total_time,ingredients,instructions,servings,category,prep_time,cook_time
0,"Desert clatite fragede, cu mere in aluat, umpl...",PT90M,\r\n 3 mere,"Se spală merele, se curăță de coajă și se tai...",9 portii,Prajituri,,
1,"Desert rulada cu portocale, frisca si crema de...",PT60M,\r\n Pentru blat:\r\n \r\n 6 oua,Blatul: Separati ouale. Albusurile le bateti ...,12 portii,Prajituri,,
2,Desert jeleuri de lamaie,PT15M,\r\n 3 - 4 lamai,\r\n Se pune intr-o cana gelatina cu 50 ...,10 portii,Dulciuri,,
3,"Desert tort magic cu mere, de post",PT90M,"\r\n Pentru blat, forma de 23 cm:\r\n \r...",Blatul: \r\nIntr-un bol amestecati faina cern...,14 portii,Dulciuri,,
4,"Musaca de cartofi, cu carne tocata, de porc",PT90M,\r\n 1 kg cartofi,"Mai întâi punem cartofii la fiert, deoarece d...",6 portii,Porc,,
...,...,...,...,...,...,...,...,...
195,Ciorba de pui cu smantana,PT60M,\r\n carne de pui,"Mai intai fierbem carnea de pui, apoi o taiem...",6 portii,Ciorbe,,
196,Varza dulce cu costita afumata,PT90M,\r\n 1 varza mare,"\r\n Mai intai tocam varza, o punem intr-o c...",6 portii,Mancaruri de legume,,
197,Desert rulada cu crema de ricotta si ciocolata,PT90M,Pt. blat: \r\n\r\n 4 oua,Blatul: Se separa ouale. Albusurile se bat ...,1 portii,Prajituri,,
198,"Desert prajitura cu mere, la tigaie",PT60M,\r\n 1 măr,Tigaia se unge cu unt şi se tapetează cu 1 li...,8 portii,Prajituri,,


In [9]:
result_df.to_csv('data/romania/5_pages_romania.csv')